# Explicatii

## Proprocesare date
###    Dupa ce am citit setul de date am analizat datele pentru a vedea daca avem celule fara informatii utile (Nan), am observat ca sunt 0.13% din celule sunt goale, iar 0.41% din totalul numarului de randuri contin cel putin o celula fara informatie. Avand in vedere procentul mic de randuri care au campuri goale, am considerat ca excluderea lor este cea mai buna varianta. Apoi am creat o noua coloana 'text' in care am concatenat toate coloanele cu informatii utile, am creat un dictionar de sinonime pentru a inlocui anumite cuvinte cu niste etichete standard si am creat etchete pe baza fisierului 'insurance_taxonomy.csv'. Etichetele sunt alcatuite din unigrame si bigrame care au fost create prin luarea fiecarei celule din colana 'labels'si impartirea acesteia in cuvinte. In urma acestui proces am atribuit cel putin o eticheta randurior din fisierul 'date.csv'.

## Antrenare 
###    Am inceput prin folosirea unui MultiLabelBinarizer pentru a transforma lista de etichete pentru fiecare rând într-o reprezentare numerică, necesară pentru antrenarea modelelor. Fiecare eticheta devine o coloana, iar fiecare rand este marcat cu 1 sau 0 in functie de prezenta etichetei. Textul brut este apoi impărtit in set de antrenament si test folosind train_test_split, astfel incât modelul sa poata fi evaluat corect pe date care nu au fost vazute anterior. Pentru a transforma textul in vectori numerici, se aplica TfidfVectorizer, care extrage cele mai relevante cuvinte si bigrame din text si creeaza o matrice numerica in care fiecare coloana corespunde unui termen, iar valorile TF-IDF masoara importanta fiecarui termen in fiecare exemplu. Modelul ales pentru clasificare este LogisticRegression, l-am aplica impreuna cu strategia One-Vs-Rest. Pentru fiecare eticheta posibila se antreneaza un model logistic separat care decide daca textul apartine sau nu acelei etichete. Modelul este antrenat pe matricea TF-IDF corespunzătoare textelor de antrenament si pe etichetele binarizate. Dupa antrenare, modelul prezice etichetele pentru setul de test, iar rezultatul este evaluat folosind classification_report.


## Rezultate
### In urma analizei rezultatelor am observat ca modelul are performante bune pentru clasele cu exemplare mai multe ceea ce era de asteptat, iar clasele mici au rezultate slabe sau foarte slabe. pentru rezultate mai bune cred ca ar trebui sa adaug mult mai multe sinonime in dictionar si probabil oversampling pentru clasele mai rare.


In [29]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os



The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [55]:
df = pd.read_csv("/kaggle/input/veridion-antrenare/date.csv")
labels = pd.read_csv("/kaggle/input/veridion-antrenare/insurance_taxonomy.csv")


In [56]:
df.head(3)

,description,business_tags,sector,category,niche
0,Welchcivils is a civil engineering and constru...,"['Construction Services', 'Multi-utilities', '...",Services,Civil Engineering Services,Other Heavy and Civil Engineering Construction
1,"Kyoto Vegetable Specialists Uekamo, also known...","['Wholesale', 'Dual-task Movement Products', '...",Manufacturing,Fruit & Vegetable - Markets & Stores,"Frozen Fruit, Juice, and Vegetable Manufacturing"
2,Loidholdhof Integrative Hofgemeinschaft is a c...,"['Living Forms', 'Farm Cafe', 'Fresh Coffee', ...",Manufacturing,Farms & Agriculture Production,All Other Miscellaneous Crop Farming


In [57]:
df.describe()

,description,business_tags,sector,category,niche
count,9482,9494,9467,9467,9494
unique,9477,9062,7,450,957
top,The company is a business analysis and English...,[],Manufacturing,Farms & Agriculture Production,Other Heavy and Civil Engineering Construction
freq,3,363,4005,237,10


In [58]:
df.isna().sum()

description      12
business_tags     0
sector           27
category         27
niche             0
dtype: int64

In [59]:
total_cells = np.product(df.shape)
total_missing = df.isnull().sum().sum()
percent_missing = (total_missing/total_cells) * 100
print(f'{percent_missing:.3f}% din celule sunt goale')
#0.139%din celule sunt goale

0.139% din celule sunt goale


In [60]:
total_rows = df.shape[0]
total_missing_rows = df.isna().any(axis=1).sum()
percent_missing_rows = (total_missing_rows/total_rows) * 100
print(f'{percent_missing_rows:.3f}% din randuri au cel putin o celula goala care ar inflenta negativ antrenarea')
# 0.41% din randuri au cel putin o celula cu Nan sau None

0.411% din randuri au cel putin o celula goala care ar inflenta negativ antrenarea


In [61]:
df[df.isna().any(axis=1)].index


Index([ 207,  607,  963, 1032, 1102, 1210, 1917, 2137, 2270, 2321, 2941, 3449,
       3910, 4017, 4399, 4544, 5152, 5198, 5345, 5506, 5660, 6098, 6285, 6287,
       6349, 6510, 6950, 6956, 7102, 7427, 7584, 7738, 7920, 7991, 8058, 8872,
       9043, 9079, 9246],
      dtype='int64')

In [62]:
df_del=df.dropna().copy()
df_del.describe()

,description,business_tags,sector,category,niche
count,9455,9455,9455,9455,9455
unique,9450,9024,7,450,956
top,The company is a business analysis and English...,[],Manufacturing,Farms & Agriculture Production,Other Heavy and Civil Engineering Construction
freq,3,362,4004,237,10


In [63]:
text_cols = ["description", "business_tags", "sector", "category", "niche" ]  
for c in text_cols:
    if c not in df_del.columns:
        df[c] = ""

df_del


,description,business_tags,sector,category,niche
0,Welchcivils is a civil engineering and constru...,"['Construction Services', 'Multi-utilities', '...",Services,Civil Engineering Services,Other Heavy and Civil Engineering Construction
1,"Kyoto Vegetable Specialists Uekamo, also known...","['Wholesale', 'Dual-task Movement Products', '...",Manufacturing,Fruit & Vegetable - Markets & Stores,"Frozen Fruit, Juice, and Vegetable Manufacturing"
2,Loidholdhof Integrative Hofgemeinschaft is a c...,"['Living Forms', 'Farm Cafe', 'Fresh Coffee', ...",Manufacturing,Farms & Agriculture Production,All Other Miscellaneous Crop Farming
3,PATAGONIA Chapa Y Pintura is an auto body shop...,"['Automotive Body Repair Services', 'Interior ...",Services,Auto Body Shops,"Automotive Body, Paint, and Interior Repair an..."
4,Stanica WODNA PTTK Swornegacie is a cultural e...,"['Cultural Activities', 'Accommodation Service...",Services,Boat Tours & Cruises,"Scenic and Sightseeing Transportation, Water"
...,...,...,...,...,...
9489,"Anhui Zhongxin Electric Co., Ltd. is a high-te...","['Automation Equipment', 'Technical Consulting...",Manufacturing,Electric Supplies & Power Generation,All Other Miscellaneous Electrical Equipment a...
9490,"TP Material Co.,Ltd. is a company based in the...","['Construction Materials Supplier', 'Construct...",Services,Construction Services,Commercial and Institutional Building Construc...
9491,Aladiner Cherag is a company that offers a var...,"['Fruit And Vegetables', 'Hand Wash Products',...",Manufacturing,Dairy Products - Farms & Stores,Fluid Milk Manufacturing
9492,Candor Eeg is a medical care company located i...,"['Stress Tests', 'Hyperventilation', 'Holter M...",Services,Radiology Clinic,Diagnostic Imaging Centers


In [64]:
df_del["original_index"] = df_del.index
df_del

,description,business_tags,sector,category,niche,original_index
0,Welchcivils is a civil engineering and constru...,"['Construction Services', 'Multi-utilities', '...",Services,Civil Engineering Services,Other Heavy and Civil Engineering Construction,0
1,"Kyoto Vegetable Specialists Uekamo, also known...","['Wholesale', 'Dual-task Movement Products', '...",Manufacturing,Fruit & Vegetable - Markets & Stores,"Frozen Fruit, Juice, and Vegetable Manufacturing",1
2,Loidholdhof Integrative Hofgemeinschaft is a c...,"['Living Forms', 'Farm Cafe', 'Fresh Coffee', ...",Manufacturing,Farms & Agriculture Production,All Other Miscellaneous Crop Farming,2
3,PATAGONIA Chapa Y Pintura is an auto body shop...,"['Automotive Body Repair Services', 'Interior ...",Services,Auto Body Shops,"Automotive Body, Paint, and Interior Repair an...",3
4,Stanica WODNA PTTK Swornegacie is a cultural e...,"['Cultural Activities', 'Accommodation Service...",Services,Boat Tours & Cruises,"Scenic and Sightseeing Transportation, Water",4
...,...,...,...,...,...,...
9489,"Anhui Zhongxin Electric Co., Ltd. is a high-te...","['Automation Equipment', 'Technical Consulting...",Manufacturing,Electric Supplies & Power Generation,All Other Miscellaneous Electrical Equipment a...,9489
9490,"TP Material Co.,Ltd. is a company based in the...","['Construction Materials Supplier', 'Construct...",Services,Construction Services,Commercial and Institutional Building Construc...,9490
9491,Aladiner Cherag is a company that offers a var...,"['Fruit And Vegetables', 'Hand Wash Products',...",Manufacturing,Dairy Products - Farms & Stores,Fluid Milk Manufacturing,9491
9492,Candor Eeg is a medical care company located i...,"['Stress Tests', 'Hyperventilation', 'Holter M...",Services,Radiology Clinic,Diagnostic Imaging Centers,9492


In [65]:
    df_del["text"] =(df_del["description"].fillna("") + " " +
                df_del["business_tags"].fillna("") + " " +
                df_del["sector"].fillna("") + " " +
                df_del['category'].fillna("") + " " +
                df_del['niche']).str.strip().str.lower()
df_del = df_del[df_del["text"].str.len() > 0].reset_index(drop=True)
print("Rows after dropping empty text:", len(df_del))

Rows after dropping empty text: 9455


In [66]:
df_del['text']

0       welchcivils is a civil engineering and constru...
1       kyoto vegetable specialists uekamo, also known...
2       loidholdhof integrative hofgemeinschaft is a c...
3       patagonia chapa y pintura is an auto body shop...
4       stanica wodna pttk swornegacie is a cultural e...
                              ...                        
9450    anhui zhongxin electric co., ltd. is a high-te...
9451    tp material co.,ltd. is a company based in the...
9452    aladiner cherag is a company that offers a var...
9453    candor eeg is a medical care company located i...
9454    reka vertrieb deutschland is a company based i...
Name: text, Length: 9455, dtype: object

In [67]:

df_del["text_len"] = df_del["text"].str.len()
print(df_del["text_len"].describe())

dups = df_del.duplicated(subset=["text"]).sum()
print("Duplicates:", dups)


df_del.sample(5)[["business_tags", "text_len", "text"]]

count    9455.000000
mean      818.066103
std       373.932595
min       101.000000
25%       518.000000
50%       830.000000
75%      1081.000000
max      2567.000000
Name: text_len, dtype: float64
Duplicates: 2


,business_tags,text_len,text
4341,"['Smart Shops Operators', 'Security Systems In...",336,"mohamed damsoth, also known as محمد دمسوس, is ..."
5217,"['Research and Analysis', 'Pharmaceutical Rese...",1286,philpharm is a research project funded by the ...
207,"['Massage Chairs', 'Sauna and Fireplace Manufa...",557,sunheat international is a company based in gr...
4561,"['Financial Projections Services', 'Accounting...",777,seguros e contabilidades is a consulting firm ...
1044,"['Watermarking for Digital Color Images', 'Mar...",1246,"zhiwei xu is a computer vision, machine learni..."


In [70]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [81]:
lemmatizer = WordNetLemmatizer()

synonyms = {
    # Veterinary
    "vet": "veterinary services",
    "vet clinic": "veterinary services",
    "veterinary clinic": "veterinary services",
    
    # Volunteer
    "volunteer": "volunteer services",
    "charity work": "volunteer services",
    "nonprofit": "volunteer services",
    
    # Waterproofing
    "waterproof": "waterproofing services",
    "roof seal": "waterproofing services",
    
    # Steam
    "steam clean": "steam services",
    "steam": "steam services",
    
    # Stakeholder
    "stakeholder": "stakeholder services",
    "stakeholder engagement": "stakeholder services",
    
    # Septic System
    "septic": "septic system services",
    "sewage tank": "septic system services",
    
    # SEO
    "seo": "seo services",
    "search engine optimization": "seo services",
    
    # Rendering
    "render": "rendering services",
    "3d render": "rendering services",
    
    # Publishing
    "publish": "publishing services",
    "printing": "publishing services",
    
    # Physical Therapy
    "physiotherapy": "physical therapy services",
    "pt": "physical therapy services",
    
    # Pesticide Application
    "pesticide": "pesticide application services",
    "herbicide": "pesticide application services",
    
    # Millwork
    "millwork": "millwork services",
    "woodwork": "millwork services",
    
    # Landscaping
    "landscape": "landscaping services",
    "gardening": "landscaping services",
    
    # Infrastructure Excavation / Excavation
    "excavation": "excavation services",
    "dig": "excavation services",
    "earthwork": "excavation services",
    
    # HVAC Inspections
    "hvac": "hvac inspections",
    "heating ventilation air conditioning": "hvac inspections",
    
    # Fundraising
    "fundraise": "fundraising services",
    "donation drive": "fundraising services",
    
    # Fishing and Hunting
    "fishing": "fishing and hunting services",
    "hunting": "fishing and hunting services",
    
    # Drywall
    "drywall": "drywall services",
    "gypsum board": "drywall services",
    
    # E-Commerce
    "ecommerce": "e-commerce services",
    "online store": "e-commerce services",
    
    # Distilling
    "distill": "distilling services",
    "distillery": "distilling services",
    
    # Compliance
    "compliance": "compliance services",
    "regulation": "compliance services",
    
    # Carpentry
    "carpenter": "carpentry services",
    "woodwork": "carpentry services",
    
    # Canning
    "can": "canning services",
    "food canning": "canning services",
    
    # Branding
    "brand": "branding services",
    "logo design": "branding services"
}

In [84]:
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
labels = pd.read_csv("/kaggle/input/veridion-antrenare/insurance_taxonomy.csv")["label"].astype(str).tolist()
custom_stopwords = set(["services",'company', 'solutions' ])
def label_to_keywords(label):
    s = label.lower()
    s = re.sub(r'[^a-z0-9\s]', ' ', s)
    tokens = [t for t in s.split() if t not in ENGLISH_STOP_WORDS and t not in custom_stopwords and len(t) > 2]
    
    # lematizare
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    
    # aplicare synonyms mapping
    tokens = [synonyms.get(t, t) for t in tokens]
    
    bigrams = [" ".join(tokens[i:i+2]) for i in range(len(tokens)-1)]
    return list(dict.fromkeys(tokens + bigrams))

In [85]:
rules = {lab: label_to_keywords(lab) for lab in labels}
# print prime 10
for lab in labels[:10]:
    print(lab, "->", rules[lab][:6])

Agricultural Equipment Services -> ['agricultural', 'equipment', 'agricultural equipment']
Soil Nutrient Application Services -> ['soil', 'nutrient', 'application', 'soil nutrient', 'nutrient application']
Pesticide Application Services -> ['pesticide application services', 'application', 'pesticide application services application']
Ornamental Plant Nurseries -> ['ornamental', 'plant', 'nursery', 'ornamental plant', 'plant nursery']
Landscaping Services -> ['landscaping']
Gardening Services -> ['landscaping services']
Tree Services - Pruning / Removal -> ['tree', 'pruning', 'removal', 'tree pruning', 'pruning removal']
Veterinary Services -> ['veterinary']
Veterinary Clinics -> ['veterinary', 'clinic', 'veterinary clinic']
Pet Boarding Services -> ['pet', 'boarding', 'pet boarding']


In [86]:

def apply_rules(text):
    assigned = []
    text = text.lower()
    for label, kws in rules.items():
        for kw in kws:
            if kw in text:
                assigned.append(label)
                break
    return assigned

df_del["weak_labels_rules"] = df_del["text"].apply(apply_rules)
coverage_rules = (df_del["weak_labels_rules"].map(len) > 0).mean()
print("Coverage (rule-based):", coverage_rules)

Coverage (rule-based): 0.997567424643046


In [87]:
df_del.sample(5)[[ "text", "weak_labels_rules"]]

,text,weak_labels_rules
3975,ck day trips is a transportation service compa...,"[Animal Day Care Services, HVAC Installation a..."
2950,golden tyres is a business located in vaijapur...,"[HVAC Installation and Service, Air Duct Clean..."
6709,pawlicy advisor is a company that was co-found...,"[Soil Nutrient Application Services, Pesticide..."
7596,alberta that specializes in architecture and d...,"[Residential Driveway Construction, Commercial..."
1732,gorleston mot & service centre is a car inspec...,"[Well Maintenance Services, General Snow Remov..."


In [88]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(df_del["weak_labels_rules"])

In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X = df_del["text"]
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

tfidf = TfidfVectorizer(max_features=20000, ngram_range=(1,2))
X_train_vec = tfidf.fit_transform(X_train)
X_test_vec = tfidf.transform(X_test)

In [90]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

clf = OneVsRestClassifier(LogisticRegression(max_iter=300))
clf.fit(X_train_vec, y_train)

y_pred = clf.predict(X_test_vec)

# Evaluare simplă
print(classification_report(y_test, y_pred, target_names=mlb.classes_))

                                                  precision    recall  f1-score   support

                         Accessory Manufacturing       0.98      0.94      0.96       865
                            Advertising Services       1.00      0.22      0.36        23
                 Agricultural Equipment Services       0.98      0.62      0.76       371
             Agricultural Machinery Installation       0.99      0.51      0.67       284
                      Air Duct Cleaning Services       0.89      0.91      0.90      1196
                     Alarm Installation Services       1.00      0.33      0.50       148
                        Animal Day Care Services       1.00      0.35      0.52       332
                       Animal Feed Manufacturing       0.97      0.93      0.95       894
                                 Animal Trainers       1.00      0.34      0.51        71
                        Animal Training Services       1.00      0.36      0.53       168
         

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.

In [91]:
df_del


,description,business_tags,sector,category,niche,original_index,text,text_len,weak_labels_rules
0,Welchcivils is a civil engineering and constru...,"['Construction Services', 'Multi-utilities', '...",Services,Civil Engineering Services,Other Heavy and Civil Engineering Construction,0,welchcivils is a civil engineering and constru...,1114,"[Residential Driveway Construction, Commercial..."
1,"Kyoto Vegetable Specialists Uekamo, also known...","['Wholesale', 'Dual-task Movement Products', '...",Manufacturing,Fruit & Vegetable - Markets & Stores,"Frozen Fruit, Juice, and Vegetable Manufacturing",1,"kyoto vegetable specialists uekamo, also known...",1201,"[Agricultural Equipment Services, Ornamental P..."
2,Loidholdhof Integrative Hofgemeinschaft is a c...,"['Living Forms', 'Farm Cafe', 'Fresh Coffee', ...",Manufacturing,Farms & Agriculture Production,All Other Miscellaneous Crop Farming,2,loidholdhof integrative hofgemeinschaft is a c...,946,"[High-Rise Signage Installation, High-Rise Gla..."
3,PATAGONIA Chapa Y Pintura is an auto body shop...,"['Automotive Body Repair Services', 'Interior ...",Services,Auto Body Shops,"Automotive Body, Paint, and Interior Repair an...",3,patagonia chapa y pintura is an auto body shop...,318,"[Well Maintenance Services, Road Maintenance S..."
4,Stanica WODNA PTTK Swornegacie is a cultural e...,"['Cultural Activities', 'Accommodation Service...",Services,Boat Tours & Cruises,"Scenic and Sightseeing Transportation, Water",4,stanica wodna pttk swornegacie is a cultural e...,496,"[Veterinary Health Centers, Land Leveling Serv..."
...,...,...,...,...,...,...,...,...,...
9450,"Anhui Zhongxin Electric Co., Ltd. is a high-te...","['Automation Equipment', 'Technical Consulting...",Manufacturing,Electric Supplies & Power Generation,All Other Miscellaneous Electrical Equipment a...,9489,"anhui zhongxin electric co., ltd. is a high-te...",1250,"[Agricultural Equipment Services, Commercial I..."
9451,"TP Material Co.,Ltd. is a company based in the...","['Construction Materials Supplier', 'Construct...",Services,Construction Services,Commercial and Institutional Building Construc...,9490,"tp material co.,ltd. is a company based in the...",368,"[Well Maintenance Services, Residential Drivew..."
9452,Aladiner Cherag is a company that offers a var...,"['Fruit And Vegetables', 'Hand Wash Products',...",Manufacturing,Dairy Products - Farms & Stores,Fluid Milk Manufacturing,9491,aladiner cherag is a company that offers a var...,732,"[Pet Boarding Services, Animal Day Care Servic..."
9453,Candor Eeg is a medical care company located i...,"['Stress Tests', 'Hyperventilation', 'Holter M...",Services,Radiology Clinic,Diagnostic Imaging Centers,9492,candor eeg is a medical care company located i...,964,"[Soil Nutrient Application Services, Pesticide..."


In [93]:
#df = df.drop("insurance_label", axis=1)


In [94]:
df["id"] = df.index



In [95]:
df.insert(0, "id", df.pop("id"))





In [96]:
df["insurance_label"] = df["id"].map(df_del.set_index("original_index")["weak_labels_rules"])
df

,id,description,business_tags,sector,category,niche,insurance_label
0,0,Welchcivils is a civil engineering and constru...,"['Construction Services', 'Multi-utilities', '...",Services,Civil Engineering Services,Other Heavy and Civil Engineering Construction,"[Residential Driveway Construction, Commercial..."
1,1,"Kyoto Vegetable Specialists Uekamo, also known...","['Wholesale', 'Dual-task Movement Products', '...",Manufacturing,Fruit & Vegetable - Markets & Stores,"Frozen Fruit, Juice, and Vegetable Manufacturing","[Agricultural Equipment Services, Ornamental P..."
2,2,Loidholdhof Integrative Hofgemeinschaft is a c...,"['Living Forms', 'Farm Cafe', 'Fresh Coffee', ...",Manufacturing,Farms & Agriculture Production,All Other Miscellaneous Crop Farming,"[High-Rise Signage Installation, High-Rise Gla..."
3,3,PATAGONIA Chapa Y Pintura is an auto body shop...,"['Automotive Body Repair Services', 'Interior ...",Services,Auto Body Shops,"Automotive Body, Paint, and Interior Repair an...","[Well Maintenance Services, Road Maintenance S..."
4,4,Stanica WODNA PTTK Swornegacie is a cultural e...,"['Cultural Activities', 'Accommodation Service...",Services,Boat Tours & Cruises,"Scenic and Sightseeing Transportation, Water","[Veterinary Health Centers, Land Leveling Serv..."
...,...,...,...,...,...,...,...
9489,9489,"Anhui Zhongxin Electric Co., Ltd. is a high-te...","['Automation Equipment', 'Technical Consulting...",Manufacturing,Electric Supplies & Power Generation,All Other Miscellaneous Electrical Equipment a...,"[Agricultural Equipment Services, Commercial I..."
9490,9490,"TP Material Co.,Ltd. is a company based in the...","['Construction Materials Supplier', 'Construct...",Services,Construction Services,Commercial and Institutional Building Construc...,"[Well Maintenance Services, Residential Drivew..."
9491,9491,Aladiner Cherag is a company that offers a var...,"['Fruit And Vegetables', 'Hand Wash Products',...",Manufacturing,Dairy Products - Farms & Stores,Fluid Milk Manufacturing,"[Pet Boarding Services, Animal Day Care Servic..."
9492,9492,Candor Eeg is a medical care company located i...,"['Stress Tests', 'Hyperventilation', 'Holter M...",Services,Radiology Clinic,Diagnostic Imaging Centers,"[Soil Nutrient Application Services, Pesticide..."
